In [0]:
# ==============================================================================
# CONFIGURAÇÕES E IMPORTS
# ==============================================================================
import pandas as pd
from pyspark.sql import functions as F, Window
from pyspark.sql.types import DoubleType

# Configurar o uso do nosso catálogo e dos schemas
spark.sql("USE CATALOG previsao_brasileirao")

silver_schema = "silver"
gold_schema = "gold"

print(f"Lendo dados de: {silver_schema}")
print(f"Salvando dados em: {gold_schema}")

In [0]:
# ==============================================================================
# PASSO 1: CALCULAR O RATING ELO PARA CADA TIME A CADA RODADA
# ==============================================================================
print("Iniciando o cálculo do Rating Elo...")

# Puxar as partidas da camada Silver e ordenar cronologicamente
df_partidas = spark.table(f"{silver_schema}.partidas").orderBy("data_partida")
partidas_pd = df_partidas.toPandas()

# Configurações do Elo
K_FACTOR = 32
ELO_INICIAL = 1500
elos = {} # Dicionário para armazenar o Elo atual de cada time

# Listas para armazenar os Elos pré-partida
elos_mandante_pre_jogo = []
elos_visitante_pre_jogo = []

# Iterar sobre cada partida em ordem para calcular o Elo
for index, row in partidas_pd.iterrows():
    mandante_id = row['mandante_id']
    visitante_id = row['visitante_id']
    resultado = row['resultado']
    
    # Inicializar o Elo dos times se for a primeira vez que os vemos
    elo_mandante = elos.get(mandante_id, ELO_INICIAL)
    elo_visitante = elos.get(visitante_id, ELO_INICIAL)
    
    # Armazenar os Elos ANTES da partida
    elos_mandante_pre_jogo.append(elo_mandante)
    elos_visitante_pre_jogo.append(elo_visitante)
    
    # Calcular a probabilidade de vitória esperada para o mandante
    prob_mandante = 1 / (1 + 10**((elo_visitante - elo_mandante) / 400))
    
    # Determinar o resultado da partida (S)
    if resultado == 'VITORIA_MANDANTE':
        score_mandante = 1.0
    elif resultado == 'EMPATE':
        score_mandante = 0.5
    else: # VITORIA_VISITANTE
        score_mandante = 0.0
        
    # Calcular o novo Elo para o mandante
    novo_elo_mandante = elo_mandante + K_FACTOR * (score_mandante - prob_mandante)
    # A mudança no Elo do visitante é o inverso da do mandante
    novo_elo_visitante = elo_visitante + K_FACTOR * ((1 - score_mandante) - (1 - prob_mandante))
    
    # Atualizar o dicionário de Elos para a próxima rodada
    elos[mandante_id] = novo_elo_mandante
    elos[visitante_id] = novo_elo_visitante

# Adicionar as novas colunas de Elo ao DataFrame Pandas
partidas_pd['elo_mandante_pre_jogo'] = elos_mandante_pre_jogo
partidas_pd['elo_visitante_pre_jogo'] = elos_visitante_pre_jogo
partidas_pd['elo_diff'] = partidas_pd['elo_mandante_pre_jogo'] - partidas_pd['elo_visitante_pre_jogo']

# Converter de volta para um DataFrame Spark
df_elos = spark.createDataFrame(partidas_pd[['partida_id', 'elo_mandante_pre_jogo', 'elo_visitante_pre_jogo', 'elo_diff']])

print("✅ Cálculo do Elo concluído.")
display(df_elos.limit(10))

In [0]:
# ==============================================================================
# PASSO 2: AGREGAR SCOUTS POR TIME E PARTIDA
# ==============================================================================
print("Agregando scouts por time e partida...")

df_stats_jogador = spark.table(f"{silver_schema}.estatisticas_jogador_partida")

# Agrupar por partida e clube para somar todos os scouts
df_stats_time_partida = df_stats_jogador.groupBy("partida_id", "clube_id").agg(
    F.sum("G").alias("gols"),
    F.sum("A").alias("assistencias"),
    (F.sum("FF") + F.sum("FD") + F.sum("G")).alias("finalizacoes_totais"),
    F.sum("DS").alias("desarmes"),
    F.sum("FC").alias("faltas_cometidas"),
    F.sum("FS").alias("faltas_sofridas"),
    F.sum("PE").alias("passes_errados")
    # Adicione outros scouts que julgar relevantes aqui
)

print("✅ Agregação de scouts concluída.")
display(df_stats_time_partida.limit(10))

In [0]:
# ==============================================================================
# PASSO 3: CALCULAR MÉDIAS MÓVEIS DE PERFORMANCE
# ==============================================================================
print("Calculando médias móveis de performance (últimos 5 jogos)...")

# Precisamos juntar as estatísticas com a data da partida para ordenação
df_partidas_base = spark.table(f"{silver_schema}.partidas").select("partida_id", "data_partida", "mandante_id", "visitante_id")
df_stats_com_data = df_stats_time_partida.join(df_partidas_base, "partida_id")

# Criar uma tabela com uma linha por time por partida (desnormalizando mandante/visitante)
mandantes = df_stats_com_data.withColumnRenamed("mandante_id", "clube_id_ref").drop("visitante_id")
visitantes = df_stats_com_data.withColumnRenamed("visitante_id", "clube_id_ref").drop("mandante_id")
df_times_partidas = mandantes.unionByName(visitantes).filter(F.col("clube_id") == F.col("clube_id_ref")).drop("clube_id_ref")

# Definir a Window Spec: para cada time, ordenar por data e olhar as 5 linhas anteriores
window_spec = Window.partitionBy("clube_id").orderBy("data_partida").rowsBetween(-5, -1)

# Calcular as médias móveis
df_medias_moveis = df_times_partidas.withColumn("media_movel_gols_marcados", F.avg("gols").over(window_spec)) \
                                    .withColumn("media_movel_finalizacoes", F.avg("finalizacoes_totais").over(window_spec)) \
                                    .withColumn("media_movel_desarmes", F.avg("desarmes").over(window_spec))

# Selecionar as colunas que precisamos
df_features_forma = df_medias_moveis.select(
    "partida_id",
    "clube_id",
    "media_movel_gols_marcados",
    "media_movel_finalizacoes",
    "media_movel_desarmes"
)

# Substituir nulos por 0 (para os primeiros jogos da temporada de um time)
df_features_forma = df_features_forma.na.fill(0)

print("✅ Cálculo de médias móveis concluído.")
display(df_features_forma.limit(10))

In [0]:
# ==============================================================================
# PASSO 4: MONTAR A TABELA FINAL 'feature_store'
# ==============================================================================
print("Montando a tabela final feature_store...")

# Tabela base com o resultado (nosso alvo)
df_base = spark.table(f"{silver_schema}.partidas").select("partida_id", "rodada", "mandante_id", "visitante_id", "resultado")

# Juntar features de Elo
df_com_elo = df_base.join(df_elos, "partida_id")

# Separar as features de forma para mandante e visitante para o join
features_mandante = df_features_forma.withColumnRenamed("clube_id", "mandante_id") \
                                     .withColumnRenamed("media_movel_gols_marcados", "mm_gols_m") \
                                     .withColumnRenamed("media_movel_finalizacoes", "mm_fin_m") \
                                     .withColumnRenamed("media_movel_desarmes", "mm_des_m")

features_visitante = df_features_forma.withColumnRenamed("clube_id", "visitante_id") \
                                      .withColumnRenamed("media_movel_gols_marcados", "mm_gols_v") \
                                      .withColumnRenamed("media_movel_finalizacoes", "mm_fin_v") \
                                      .withColumnRenamed("media_movel_desarmes", "mm_des_v")

# Juntar as features de forma do mandante
df_final = df_com_elo.join(features_mandante, ["partida_id", "mandante_id"], "left")
# Juntar as features de forma do visitante
df_final = df_final.join(features_visitante, ["partida_id", "visitante_id"], "left")

# Calcular diferenciais de "forma"
df_final = df_final.withColumn("diff_mm_gols", F.col("mm_gols_m") - F.col("mm_gols_v")) \
                   .withColumn("diff_mm_fin", F.col("mm_fin_m") - F.col("mm_fin_v")) \
                   .withColumn("diff_mm_des", F.col("mm_des_m") - F.col("mm_des_v"))

# Selecionar e ordenar as colunas finais
df_final = df_final.select(
    "partida_id", "rodada", "mandante_id", "visitante_id",
    # Features de Força
    "elo_mandante_pre_jogo", "elo_visitante_pre_jogo", "elo_diff",
    # Features de Forma (Mandante)
    "mm_gols_m", "mm_fin_m", "mm_des_m",
    # Features de Forma (Visitante)
    "mm_gols_v", "mm_fin_v", "mm_des_v",
    # Features de Diferencial
    "diff_mm_gols", "diff_mm_fin", "diff_mm_des",
    # Alvo
    "resultado"
).na.fill(0)


# Salvar a tabela final na camada Gold
df_final.write \
    .mode("overwrite") \
    .format("delta") \
    .option("overwriteSchema", "true") \
    .saveAsTable(f"{gold_schema}.feature_store")

print("✅ Tabela 'feature_store' criada com sucesso na camada Gold.")
display(spark.table(f"{gold_schema}.feature_store").limit(10))